### Setup

In [3]:
import re
import os
import pandas as pd
path_master = r'C:/Users/Ayushee/Dropbox/Columbia/Spring 2019/Analytics in Practice/Data/Csv/'
file_names = list()
for path, subdirs, files in os.walk(path_master):
    for filename in files:
        file_names.append(filename)
df_final = pd.DataFrame()

### Functions

In [4]:
def conv(x):
    x = str(x)
    x = x.strip().replace(',','')
    x = x.replace(' ','')
    if re.search(pattern = r'\w+',string=x):
        return x
    elif x=='-' or x=='0-':
        return 0
    elif re.search(pattern=r'%', string=x):
        return float(x.replace('%',''))
    elif re.search(pattern=r'$',string=x):
        return x.replace('$','').replace(',','').replace('-','0')
    else:
        return x

def add_risk(df,fn):
    """Add Risk column based on file number"""  
    if fn=='1':
            df.drop(df.index[0],inplace=True)
            df['Risk']='Low'
    if fn=='2':
            df['Risk']='Low'
    if fn=='3':
            df.drop(df.index[0],inplace=True)
            df['Risk']='Moderate'
    if fn=='4':
            df['Risk']='Moderate'
    if fn=='5':
            df.drop(df.index[0],inplace=True)
            df['Risk']='High'
    if fn=='6':
            df['Risk']='High'
    if fn=='7':
            df.drop(df.index[0],inplace=True)
            df['Risk']='Ultra High'
    if fn=='8':
            df['Risk']='Ultra High'
    if fn=='9':
            df.drop(df.index[0],inplace=True)
            df['Risk']='Natural'
    if fn=='10':
            df['Risk']='Natural'
            df.drop(df.index[-1],inplace=True)
    df.fillna(0,inplace=True)
        
    
def add_season(month):
    springs = ['February','March']
    falls = ['September','October','November','December']
    if month in springs:
        season = "Spring"
    elif month in falls:
        season = "Fall"
    else: 
        season = "Other"
    return season

def merge_cols(df,col1,col2):
    """Merge any two columns in df"""
    df[col1] = df[col1].map(lambda x: str(x).split('.')[0]) + df[col2]
    del df[col2]
    
def add_time(df_ABR1):
    """Add Month, Year, Season columns based on read-in file name"""
    mon=re.search(r'\w+', file_name)
    df_ABR1['Month']= file_name[mon.start():mon.end()]
    y=re.search(r'\d+', file_name)
    df_ABR1['Year'] = file_name[y.start():y.end()]            
    add_season(file_name[mon.start():mon.end()])
    
def type1(pm,tt,fn,d='.csv'):
    """10 columns, missing 7,9, merge 3,4"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','8','10'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx','10':'Treatment $'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type2(pm,tt,fn,d='.csv'):
    """9 columns, missing 6,8"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                              usecols=['0','1','2','3','4','5','7','9'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                            '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', '9':'Treatment $',}, 
                       inplace=True)
    df_ABR1.fillna(0,inplace=True)
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type3(pm,tt,fn,d='.csv'):
    """base case"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                          skiprows=[0], 
                          usecols=['Unnamed: 1', 'Treated', 'Repulls', 'Mortalities', 'Treatment Success',
                                   'Case Fatality Rate','$/Hd/Rx', 'Treatment $'])
    df_ABR1.rename(columns={'Unnamed: 1': 'Drug', 'Treatment Success':'Treatment Success(%)',
                        'Case Fatality Rate':'Case Fatality Rate(%)'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type4(pm,tt,fn,d='.csv'):
    """11 columns, missing 7,9, merge 2,3"""
    with open(pm+tt+fn+d) as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','8','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'RepullsB', '4':'Mortalities', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate Treatment $ column and Repulls column
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    merge_cols(df_ABR1,'Repulls','RepullsB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type5(pm,tt,fn,d='.csv'):
    """10 column, missing 6,8, merge 9,10"""
    with open(pm+tt+fn+d) as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','7','9','10'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                        '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', 
                        '9':'Treatment $','10':'Treatment $B'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate Treatment $ column
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type6(pm,tt,fn,d='.csv'):
    """11 column, missing 2, merge 4,5 and 8,9 and 10,11"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','3','4','5','6','7','8','9','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '3':'Repulls', '4':'Mortalities','5':'MortalitiesB','6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '8':'$/Hd/Rx', '9':'$/Hd/RxB',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate Treatment $ column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    merge_cols(df_ABR1,'$/Hd/Rx','$/Hd/RxB')
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    df_ABR1['Treated']=df_ABR1['Treated'].apply(lambda x: x.replace(' ',''))
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type7(pm,tt,fn,d='.csv'):
    """11 columns, missing 8,10"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','4','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '4':'Mortalities', '6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type8(pm,tt,fn,d='.csv'):
    """10 columns, missing 8,9, concatenate Mortalities column"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','7','10'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '7':'$/Hd/Rx','10':'Treatment $'},
                   inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type9(pm,tt,fn,d='.csv'):
    """13 columns, missing 10,12"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','5','6','8','9','11','13'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated','2':'Repulls','3':'RepullsB','5':'Mortalities','6':'MortalitiesB',
                            '8':'Treatment Success(%)','9':'Case Fatality Rate(%)','11':'$/Hd/Rx','13':'Treatment $'},
                   inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate Repulls & Mortalities columns
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    merge_cols(df_ABR1,'Repulls','RepullsB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type10(pm,tt,fn,d='.csv'):
    """9 columns, missing 8, merge 3,4"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','7','9'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '7':'$/Hd/Rx','9':'Treatment $'}, 
                   inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type11(pm,tt,fn,d='.csv'):
    """11 columns, missing 3,8,10 merge 4,5"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','4','5','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '4':'Mortalities', '5':'MortalitiesB',
                            '6':'Treatment Success(%)', '7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type12(pm,tt,fn,d='.csv'):
    """11 columns, missing 8,10, merge 2,3 and 4,5"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'RepullsB','4':'Mortalities','5':'MortalitiesB',
                            '6':'Treatment Success(%)','7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate Repulls & Mortalities columns
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    merge_cols(df_ABR1,'Repulls','RepullsB')
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type13(pm,tt,fn,d='.csv'):
    """11 column, missing 7,9, merge 2,3 and 10,11"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','8','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated','2':'Repulls', '3':'RepullsB', '4':'Mortalities',
                            '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)',
                            '8':'$/Hd/Rx','10':'Treatment $','11':'Treatment $B'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate Treatment $ column
    merge_cols(df_ABR1,'Repulls','RepullsB')
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    #df_ABR1['Treated']=df_ABR1['Treated'].apply(lambda x: x.replace(' ',''))
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type14(pm,tt,fn,d='.csv'):
    """9 columns, missing 7,8"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                              usecols=['0','1','2','3','4','5','6','9'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                            '5':'Case Fatality Rate(%)', '6':'$/Hd/Rx', '9':'Treatment $',}, 
                       inplace=True)
    df_ABR1.fillna(0,inplace=True)
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type15(pm,tt,fn,d='.csv'):
    """10 columns, missing 2,7,9"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','3','4','5','6','8','10'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '3':'Repulls','4':'Mortalities','5':'Treatment Success(%)', 
                            '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx','10':'Treatment $'}, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    # Concatenate mortalities column
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type16(pm,tt,fn,d='.csv'):
    """11 columns, missing 2,4,8,10"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','3','5','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '3':'Repulls', '5':'Mortalities', '6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    df_ABR1.fillna(0,inplace=True)
    add_time(df_ABR1)
    add_risk(df_ABR1,fn)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

### Clean

In [5]:
# JANUARY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'January\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '5' or file_num == '8'or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '6' or file_num == '7'or file_num == '9':
                type2(path_master,title,file_num)    

# FEBRUARY 2017 & 2018
for file_name in file_names:
    pattern = r'February\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '5'or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '4'or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '7'or file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        elif file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '5':
                type8(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '8':
                type7(path_master,title,file_num)
        
# MARCH 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'March\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3':
                type7(path_master,title,file_num)
            elif file_num == '4' or file_num == '5'or file_num == '6'or file_num == '7'or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '10':
                type9(path_master,title,file_num)
    
# APRIL 2017 & 2018
for file_name in file_names:
    pattern = r'April\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '3' or file_num == '5' or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '4' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type13(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '7':
                type6(path_master,title,file_num)
            elif file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        elif file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num=='4' or file_num=='6' or file_num=='10':
                type1(path_master,title,file_num)  
            elif file_num=='5' or file_num=='9':
                type2(path_master,title,file_num)
            elif file_num == '7':
                type15(path_master,title,file_num)
            elif file_num == '8':
                type1(path_master,title,file_num)
                
# MAY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'May\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4' or file_num == '5' or file_num == '6'or file_num == '7'or file_num == '8'or file_num == '9':
                type2(path_master,title,file_num)
                
# JUNE 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'June\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type7(path_master,title,file_num)
            elif file_num == '3' or file_num == '4' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '5' or file_num == '6' or file_num == '7'or file_num == '8':
                type2(path_master,title,file_num)
            elif file_num == '9':
                type5(path_master,title,file_num)
                
# JULY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'July\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4' or file_num == '5' or file_num == '6' or file_num == '7' or file_num == '8':
                type2(path_master,title,file_num)
            elif file_num == '3':
                type1(path_master,title,file_num)
            elif file_num == '9':
                type5(path_master,title,file_num)
            elif file_num == '10':
                type12(path_master,title,file_num)
                
# AUGUST 2017 & 2018
for file_name in file_names:
    pattern = r'August\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4' or file_num == '7' or file_num == '8':
                type2(path_master,title,file_num)
            elif file_num == '3' or file_num == '5':
                type1(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '9':
                type4(path_master,title,file_num)
            elif file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4' or file_num == '5' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '3':
                type10(path_master,title,file_num)
            elif file_num == '6' or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '10':
                type12(path_master,title,file_num)
                
# SEPTEMBER 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'September\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '3' or file_num == '5' or file_num == '8' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
                
# OCTOBER 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'October\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type12(path_master,title,file_num)
            elif file_num == '3' or file_num == '8' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type13(path_master,title,file_num)
            elif file_num == '5':
                type14(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
            
# NOVEMBER 2017 & 2018 (note: no 2017 or 2018 file)

# DECEMBER 2017 & 2018 (note: no 2018 file)
for file_name in file_names:
    pattern = r'December\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num == '4' or file_num == '8' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)

# add season column
df_final['Season'] = df_final['Month'].apply(add_season)                

# fill zeroes
df_final.fillna(0)                

# Remove percent signs from columns
df_final['Treatment Success(%)'] = df_final['Treatment Success(%)'].apply(lambda x: x[:-1])
df_final['Case Fatality Rate(%)'] = df_final['Case Fatality Rate(%)'].apply(lambda x: x[:-1])
df_final['Treatment Success(%)'] = df_final['Treatment Success(%)'].apply(lambda x: 0 if x == '' else x)

# Re-formate $columns
df_final['Treatment $'] = df_final['Treatment $'].apply(lambda x: x.strip().replace(',','').replace(' ','').replace('$',''))
df_final['$/Hd/Rx'] = df_final['$/Hd/Rx'].apply(lambda x: x.strip().replace(',','').replace(' ','').replace('$',''))

#Clean number formats
for col in df_final.columns:
    df_final[col] = df_final[col].apply(conv)

# dtypes to numeric
col_to_num = ['Treated','Repulls','Mortalities','Treatment $','$/Hd/Rx','Case Fatality Rate(%)','Treatment Success(%)']
for i in col_to_num:
    df_final[i] = df_final[i].convert_objects(convert_numeric=True).fillna(0)
df_final['Treatment Success(%)'] = df_final['Treatment Success(%)'].div(100).round(2)
df_final['Case Fatality Rate(%)'] = df_final['Case Fatality Rate(%)'].div(100).round(2)

# Datetime column
import datetime
df_final['Datetime']=df_final['Month'].astype(str)+' '+df_final['Year'].astype(str)
df_final['Datetime'] = df_final['Datetime'].apply(lambda x: datetime.datetime.strptime(x, '%B %Y'))

# Save out df_ABR
import csv
df_final.to_csv(path_master+"/df_ABR.csv")

C:\Users\Ayushee\Anaconda3\lib\site-packages\ipykernel_launcher.py:282: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


### Final df

In [6]:
import pandas as pd
with open(path_master+'df_ABR.csv') as fp:
    df_final = pd.read_csv(fp)
del df_final['Unnamed: 0']
df_final.head(7)

,Drug,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $,Month,Year,Risk,Season,Datetime
0,300PROLA,0,0.0,0.0,0.00,0.00,0.00,0.00,January,2018,Low,Other,2018-01-01
1,A180,0,0.0,0.0,0.00,0.00,0.00,0.00,January,2018,Low,Other,2018-01-01
2,A180As2ndRx,0,0.0,0.0,0.00,0.00,0.00,0.00,January,2018,Low,Other,2018-01-01
3,Adspec,0,0.0,0.0,0.00,0.00,0.00,0.00,January,2018,Low,Other,2018-01-01
4,Advocin,142,23.0,14.0,0.84,0.10,36.13,5130.82,January,2018,Low,Other,2018-01-01
5,AdvocinAs2ndRx,716,1.0,118.0,1.00,0.16,28.34,20291.44,January,2018,Low,Other,2018-01-01
6,AlbonBolus,0,0.0,0.0,0.00,0.00,0.00,0.00,January,2018,Low,Other,2018-01-01


### Grouping the data 

In [45]:
def GroupColFunc(df, ind, col):
    if df[col].loc[ind] == 'January' or df[col].loc[ind] == 'February' or df[col].loc[ind] == 'March':
        return 'Q1'
    elif df[col].loc[ind] == 'April' or df[col].loc[ind] == 'May' or df[col].loc[ind] == 'June':
        return 'Q2'
    elif df[col].loc[ind] == 'July' or df[col].loc[ind] == 'August' or df[col].loc[ind] == 'September':
        return 'Q3'
    else:
        return 'Q4'

### Calculating Treated Success %

In [58]:
nrow=0
j=1
df_final_2017 = df_final[df_final['Year']==2017]
grouped=df_final_2017.groupby(df_final_2017['Drug'])
for i in grouped.size().keys():
    df_med = df_final_2017[df_final_2017['Drug']==i]
    grouped = df_med.groupby(lambda z: GroupColFunc(df_med, z, 'Month')) 
    df_prime = pd.DataFrame(grouped.sum())
    df_prime=df_prime[['Treated','Repulls','Mortalities']]
    df_prime['TreatedSuccess%']=df_prime['Treated']/(df_prime['Treated']+df_prime['Repulls'])
    diction={'Drug':i,'Q1':df_prime['TreatedSuccess%']['Q1'],'Q2':df_prime['TreatedSuccess%']['Q2'],'Q3':df_prime['TreatedSuccess%']['Q3'],'Q4':df_prime['TreatedSuccess%']['Q4']}
    if nrow==0:
        df_new=pd.DataFrame(diction,index=[0])
        nrow=1
    else:
        df_new.loc[j] = [i,df_prime['TreatedSuccess%']['Q1'], df_prime['TreatedSuccess%']['Q2'],df_prime['TreatedSuccess%']['Q3'],df_prime['TreatedSuccess%']['Q4']]
        j=j+1
df_new.fillna(0)   

,Drug,Q1,Q2,Q3,Q4
0,300PROLA,NaN,NaN,NaN,0.770992
1,A180,NaN,NaN,NaN,NaN
2,A180As2ndRx,NaN,NaN,NaN,NaN
3,Adspec,NaN,NaN,NaN,NaN
4,Advocin,0.964371,0.903137,0.890215,0.934066
5,AdvocinAs2ndRx,1.000000,1.000000,1.000000,0.992908
6,AlbonBolus,NaN,NaN,NaN,NaN
7,Baytril,0.816832,0.960641,0.805147,0.929924
8,BaytrilAs2ndRx,1.000000,1.000000,1.000000,1.000000
9,BaytrilAs3rdRx,1.000000,1.000000,1.000000,1.000000


In [62]:
nrow=0
j=1
df_final_2018 = df_final[df_final['Year']==2018]
grouped=df_final_2018.groupby(df_final_2018['Drug'])
for i in grouped.size().keys():
    df_med = df_final_2018[df_final_2018['Drug']==i]
    grouped = df_med.groupby(lambda z: GroupColFunc(df_med, z, 'Month')) 
    df_prime = pd.DataFrame(grouped.sum())
    df_prime=df_prime[['Treated','Repulls','Mortalities']]
    df_prime['TreatedSuccess%']=df_prime['Treated']/(df_prime['Treated']+df_prime['Repulls'])
    diction={'Drug':i,'Q1':df_prime['TreatedSuccess%']['Q1'],'Q2':df_prime['TreatedSuccess%']['Q2'],'Q3':df_prime['TreatedSuccess%']['Q3'],'Q4':df_prime['TreatedSuccess%']['Q4']}
    if nrow==0:
        df_new_18=pd.DataFrame(diction,index=[0])
        nrow=1
    else:
        df_new_18.loc[j] = [i,df_prime['TreatedSuccess%']['Q1'], df_prime['TreatedSuccess%']['Q2'],df_prime['TreatedSuccess%']['Q3'],df_prime['TreatedSuccess%']['Q4']]
        j=j+1
df_new_18.fillna(0)

,Drug,Q1,Q2,Q3,Q4
0,300PROLA,0.000000,0.000000,0.000000,0.000000
1,A180,0.000000,0.000000,0.000000,0.000000
2,A180As2ndRx,0.000000,0.000000,0.000000,0.000000
3,Adspec,0.000000,0.000000,0.000000,0.000000
4,Advocin,0.955556,0.935545,0.910452,0.899301
5,AdvocinAs2ndRx,0.994182,0.998694,1.000000,1.000000
6,AlbonBolus,0.000000,0.000000,0.000000,0.000000
7,Baytril,0.798921,0.952381,0.812582,0.964578
8,BaytrilAs2ndRx,0.998912,1.000000,0.965876,0.996667
9,BaytrilAs3rdRx,0.944928,0.980690,0.957245,0.711409
